In [2]:
import pandas as pd

df = pd.read_csv("C:/Users/user/ai-proj2-class4/10.data-collecting/seoul_uv_data/서울_자외선_일별최고치_2015_2017.csv", encoding='cp949')

In [3]:
df.head()

,일자,총자외선지수 일최고값,Unnamed: 2
0,2015-01-01,NaN,NaN
1,2015-01-02,NaN,NaN
2,2015-01-03,NaN,NaN
3,2015-01-04,NaN,NaN
4,2015-01-05,NaN,NaN


In [4]:
df = df.iloc[:, :2]  # 첫 2개 컬럼만 사용
df.columns = ['datetime', 'uv_max']

# 지역 컬럼 추가
df['region'] = '서울시'

In [5]:
df.head()

,datetime,uv_max,region
0,2015-01-01,NaN,서울시
1,2015-01-02,NaN,서울시
2,2015-01-03,NaN,서울시
3,2015-01-04,NaN,서울시
4,2015-01-05,NaN,서울시


In [8]:
import json
from sqlalchemy import create_engine 
from sqlalchemy.exc import SQLAlchemyError
import cryptography

with open('db-config.json', 'r') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

df['datetime'] = pd.to_datetime(df['datetime'])  # datetime 형식으로 변환

In [8]:
# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
conn = engine.connect()
# 데이터 삽입 (예외처리 포함)
try:
    df.to_sql(name='uv_ray_day', con=conn, if_exists='append', index=False)
    print("[Success] 데이터가 성공적으로 삽입되었습니다.")
except SQLAlchemyError as e:
    print("[Failure] 데이터 삽입 중 오류 발생:", str(e))

[Success] 데이터가 성공적으로 삽입되었습니다.


In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import cryptography  # pymysql 연결용 필요

# DB 설정 불러오기
with open('db-config.json', 'r') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']


engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
conn = engine.connect()


csv_files = [
    "C:/Users/user/ai-proj2-class4/10.data-collecting/seoul_uv_data/서울_자외선_일별최고치_2018_2020.csv",
    "C:/Users/user/ai-proj2-class4/10.data-collecting/seoul_uv_data/서울_자외선_일별최고치_2021_2024.csv"
]


for file_path in csv_files:
    try:
        df = pd.read_csv(file_path, encoding='cp949')
        df = df.iloc[:, :2]  # 앞의 두 컬럼만 사용
        df.columns = ['datetime', 'uv_max']
        df['region'] = '서울시'
        df['datetime'] = pd.to_datetime(df['datetime'])

        # 삽입
        df.to_sql(name='uv_ray_day', con=conn, if_exists='append', index=False)
        print(f"[Success] {file_path} 데이터 삽입 완료.")
    except SQLAlchemyError as e:
        print(f"[Failure] {file_path} 삽입 중 오류 발생:", str(e))
    except Exception as e:
        print(f"[Error] 예외 발생 - {file_path}: {str(e)}")

conn.close()

[Success] C:/Users/user/ai-proj2-class4/10.data-collecting/seoul_uv_data/서울_자외선_일별최고치_2018_2020.csv 데이터 삽입 완료.
[Success] C:/Users/user/ai-proj2-class4/10.data-collecting/seoul_uv_data/서울_자외선_일별최고치_2021_2024.csv 데이터 삽입 완료.


In [6]:
import pandas as pd
from sqlalchemy import create_engine
import json

# 접속 정보 로딩 (옵션: db-config.json 파일이 있을 경우)
with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# weather 테이블 데이터 읽어오기
query = "SELECT * FROM air_quality;"
df = pd.read_sql(query, con=engine)

# ▶ 결과 확인
print(df)

              id            datetime region   pm10  pm25
0              1 2015-12-31 23:00:00     평균   93.0  62.0
1              2 2015-12-31 23:00:00    강남구  107.0  70.0
2              3 2015-12-31 23:00:00    강동구   91.0  51.0
3              4 2015-12-31 23:00:00    강북구   92.0  64.0
4              5 2015-12-31 23:00:00    강서구   96.0  51.0
...          ...                 ...    ...    ...   ...
2958921  2958922 2024-01-01 00:00:00    용산구   35.0  25.0
2958922  2958923 2024-01-01 00:00:00    은평구   21.0  15.0
2958923  2958924 2024-01-01 00:00:00    종로구   28.0  26.0
2958924  2958925 2024-01-01 00:00:00     중구   26.0  26.0
2958925  2958926 2024-01-01 00:00:00    중랑구   28.0  23.0

[2958926 rows x 5 columns]


In [4]:
import pandas as pd
from functools import reduce

# 파일명:컬럼명 매핑
file_map = {
    "오존 2015-2025.csv": "o3",
    "아황산가스 2015-2025.csv": "so2",
    "이산화질소 2015-2025.csv": "no2",
    "일산화탄소 2015-2025.csv": "co"
}

base_path = r"C:\Users\user\ai-proj2-class4\10.data-collecting\대기오염물질_data"
dfs = []

# 24:00 처리 함수
def fix_24h(df):
    mask = df['datetime'].str.contains("24:00")
    df.loc[mask, 'datetime'] = pd.to_datetime(df.loc[mask, 'datetime'].str.replace("24:00", "00:00")) + pd.Timedelta(days=1)
    df.loc[~mask, 'datetime'] = pd.to_datetime(df.loc[~mask, 'datetime'])
    return df

for filename, col_name in file_map.items():
    file_path = f"{base_path}\\{filename}"
    df = pd.read_csv(file_path, encoding='utf-8')
    df = df.rename(columns={df.columns[0]: 'datetime'})  # 첫 번째 열을 datetime으로

    df_melted = pd.melt(df, id_vars='datetime', var_name='region', value_name=col_name)

    # datetime 24:00 처리
    df_melted = fix_24h(df_melted)

    dfs.append(df_melted)

# 데이터 병합
merged = reduce(lambda left, right: pd.merge(left, right, on=['datetime', 'region']), dfs)

# 최종 확인
print(merged.head())
print(f"[병합 완료] 총 행 수: {len(merged)}")

              datetime region      o3     so2     no2    co
0  2015-01-01 01:00:00     평균  0.0210  0.0054  0.0099  0.36
1  2015-01-01 02:00:00     평균  0.0201  0.0055  0.0096  0.37
2  2015-01-01 03:00:00     평균  0.0197  0.0054  0.0090  0.38
3  2015-01-01 04:00:00     평균  0.0203  0.0053  0.0074  0.35
4  2015-01-01 05:00:00     평균  0.0195  0.0052  0.0074  0.34
[병합 완료] 총 행 수: 2350998
